## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

**[1]** Изучим первые 15 строк массива и посмотрим на состав столбцов.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(15)

FileNotFoundError: [Errno 2] File b'/datasets/data.csv' does not exist: b'/datasets/data.csv'

**[2]** Построим гистограмму ежемесячных доходов.

In [ ]:
ax1 = plt.hist(data['total_income'], bins=int(np.sqrt(len(data))))

# N — счетчик в каждом баре (столбце), bins — нижняя граница бара
N, bins, patches = plt.hist(data['total_income'], bins=int(np.sqrt(len(data))))
                               
# Для кодирования цвета будем использовать относительную высоту
fracs = N / N.max()

# нормализуем fracs до промежутка между 0 и 1 для полноценного цветового диапазона
norm = colors.Normalize(fracs.min(), fracs.max())

# нужно пройтись циклом по полученным объектам и установить цвет для каждого в отдельности
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

ax1 = plt.xlabel('$Ежемесячный$ $доход, руб.$', fontsize=11)
ax1 = plt.ylabel('$Клиенты, чел.$', fontsize=11)
plt.title('Гистограмма ежемесячных доходов', fontsize=13)
plt.text(1000000, 1500, '$\mu=167422,\sigma=102972$', fontsize=13)
plt.show()
print(round(data['total_income'].describe()))

**[3]** Построим гистограмму возраста клиентов.

In [ ]:
plt.xticks(np.arange((data['dob_years'].min()), (data['dob_years'].max() + 5), 5.0))
ax2 = plt.hist(data['dob_years'], bins=15, color='tab:purple')

# N — счетчик в каждом баре (столбце), bins — нижняя граница бара
N, bins, patches = plt.hist(data['dob_years'], bins=15)
                               
# Для кодирования цвета будем использовать относительную высоту
fracs = N / N.max()

# нормализуем fracs до промежутка между 0 и 1 для полноценного цветового диапазона
norm = colors.Normalize(fracs.min(), fracs.max())

# нужно пройтись циклом по полученным объектам и установить цвет для каждого в отдельности
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

ax2 = plt.xlabel('$Возраст, лет$', fontsize=11)
ax2 = plt.ylabel('$Клиенты, чел.$', fontsize=11)
plt.title('Гистограмма возраста клиентов', fontsize=13)
plt.show()

**[4]** Построим гистограмму численности детей.

In [ ]:
plt.xticks(np.arange((data['children'].min()-1), (data['children'].max() + 1), 1.0))
ax3 = plt.hist(data['children'], bins=25)

# N — счетчик в каждом баре (столбце), bins — нижняя граница бара
N, bins, patches = plt.hist(data['children'], bins=25)
                               
# Для кодирования цвета будем использовать относительную высоту
fracs = N / N.max()

# нормализуем fracs до промежутка между 0 и 1 для полноценного цветового диапазона
norm = colors.Normalize(fracs.min(), fracs.max())

# нужно пройтись циклом по полученным объектам и установить цвет для каждого в отдельности
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

ax3 = plt.xlabel('$Дети, чел.$', fontsize=11)
ax3 = plt.ylabel('$Клиенты, чел.$', fontsize=11)
plt.title('Гистограмма численности детей', fontsize=13)
plt.show()

**[5]** Построим гистограмму дней трудового стажа.

In [ ]:
ax4 = plt.hist(data['days_employed'], bins=30, color='tab:blue')

# N — счетчик в каждом баре (столбце), bins — нижняя граница бара
N, bins, patches = plt.hist(data['days_employed'], bins=30)
                               
# Для кодирования цвета будем использовать относительную высоту
fracs = N / N.max()

# нормализуем fracs до промежутка между 0 и 1 для полноценного цветового диапазона
norm = colors.Normalize(fracs.min(), fracs.max())

# нужно пройтись циклом по полученным объектам и установить цвет для каждого в отдельности
for thisfrac, thispatch in zip(fracs, patches):
    color = plt.cm.viridis(norm(thisfrac))
    thispatch.set_facecolor(color)

ax4 = plt.xlabel('$Стаж, дней$', fontsize=11)
ax4 = plt.ylabel('$Клиенты, чел.$', fontsize=11)
plt.title('Гистограмма дней трудового стажа', fontsize=13)
plt.show()

Для наглядности построим точечную диаграмму зависимости трудового стажа от возраста клиента для трех классов: нулевого возраста клиента, положительного и отрицательного трудового стажа.

In [ ]:
# сделаем выборку из нулевого возраста и соотвествующих значений трудового стажа
zero_age = data.loc[data['dob_years'] == 0, 'dob_years']
days_employed_for_age_null = data.loc[data['dob_years'] == 0, 'days_employed']

# сделаем выборку из положительных значений трудового стажа и возраста клиентов (без нулевых возрастов)
age_for_positive_days_employed = data.loc[(data['days_employed'] > 0) & (data['dob_years'] != 0), 'dob_years']
positive_days_employed = data.loc[(data['days_employed'] > 0) & (data['dob_years'] != 0), 'days_employed']

# сделаем выборку из отрицательных значений трудового стажа и возраста клиентов (без нулевых возрастов)
age_for_negative_days_employed = data.loc[(data['days_employed'] < 0) & (data['dob_years'] != 0), 'dob_years']
negative_days_employed = data.loc[(data['days_employed'] < 0) & (data['dob_years'] != 0), 'days_employed']

# строим диаграмму
plt.title('Точечная диаграмма зависимости трудового стажа от возраста клиента')
plt.xlabel('Возраст клиента, лет')
plt.ylabel('Трудовой стаж, дней')
plt.scatter(zero_age, days_employed_for_age_null, 
            c = 'c', marker = '|', label = 'Стаж у клиентов с нулевым возрастом')
plt.scatter(age_for_positive_days_employed, positive_days_employed, 
            c = 'g', marker = '|', label = 'Положительный трудовой стаж')
plt.scatter(age_for_negative_days_employed, negative_days_employed, 
            c = 'r', marker = '|', label = 'Отрицательный трудовой стаж')
plt.legend()
plt.show()

### Вывод

**[1]** В файле *'data.csv'* обнаружен двумерный массив из данных о 21525 клиентах: каждый клиент описывается 12 признаками, 4 из которых относятся к количественным признакам, 8 – к категориальным или бинарным.

Существуют пропущенные значения по общему трудовому стажу (*'days_employed'*) и ежемесячному доходу (*'total_income'*) у 2174 клиентов, это около 10% от общего числа клиентов – немалая величина. Необходимо проверить, является ли отсутствие данных по трудовому стажу причиной отсутствия данных по доходам, и наоборот.

Значения уровня образования указаны то прописными буквами, то — строчными. В будущем необходимо сделать выбор уровня образования с помощью выпадающего списка, чтобы устранить человеческий фактор при наборе этого признака. В данном массиве необходимо все значения привести к нижнему регистру для единообразия.


**[2]** Серьезных аномалий в ежемесячных доходов не выявлено, если не считать пропущенные значения и асимметрии справа. Полученные на данном этапе статистики вроде матожидания, равного 167 422 руб. и стандартного отклонения в 102 972 руб., будут пересчитаны после обработки пропусков.

**[3]** В данные о возрасте клиентов попали нулевые значения, их надо будет устранить, а также обратить внимание, действительно ли ошибка вызвана человеческим фактором.


**[4]** У большинства клиентов численность детей варьируется от 0 до 4, однако, у некоторых все же детей -1 чел. или 20 чел. Эти выбросы будут заменены из предположения, что вместо 1 ребенка записали вручную -1, а под 20 детьми подразумевали все-таки только двоих.


**[5]** И, наконец, победитель количественных аномалий — дни трудового стажа клиентов. Здесь присутствуют как отрицательные значения, так и положительные (но уже масштабом в сотни лет). Требуется информация от сотрудников, выгружавших эти данные: что именно под ними подразумевается. Возможно, эти данные отображают временную дельту от базовой даты.

### Шаг 2. Предобработка данных

### Обработка пропусков

**[1]** Обработка пропущенных значений ежемесячного дохода

Заменим пустые значения в ежемесячном доходе на медианные по каждому типу занятости.

In [ ]:
# Для этого проверим, в каких типах занятости встречаются пропущенные значения доходов.
data.loc[data['total_income'].isnull(), 'income_type'].value_counts()

In [ ]:
# Найдем медианные уровни дохода для каждого типа занятости
medians_total_income = data.groupby('income_type')['total_income'].median()
round(medians_total_income)

In [ ]:
# Заменим пропущенные значения ежемесячного дохода на медианы по типам занятости.
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]

**[2]** Исправление нулевых значений в возрасте клиентов

In [ ]:
# Посмотрим, какой тип занятости имеют клиенты с ошибочным возрастом.
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

Всего их 101 человек, большая часть из которых являются сотрудниками организаций.

In [ ]:
# Найдем медианные значения возрастов клиентов для каждого типа занятости:
age_medians = data.groupby('income_type')['dob_years'].median()
age_medians

In [ ]:
# Теперь заменим нулевые значения на медианы по каждому типу занятости.
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = age_medians[6]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = age_medians[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = age_medians[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = age_medians[2]

# Проверим, все ли нули мы исправили.
data.loc[data['dob_years'] == 0, 'dob_years'].value_counts()

**[3]** Обработка пропущенных значений трудового стажа

In [ ]:
# Проверим, является ли отсутствие данных по трудовому стажу причиной отсутствия данных по доходам, и наоборот.
count = 0
for i in range(len(data)):
    if pd.isna(data.loc[i, 'days_employed']) == pd.isna(data.loc[i, 'total_income']) == True:
        count += 1
count

Действительно, значения трудового стажа пропущены тогда и только тогда, когда пропущены значения ежемесячного дохода.

Для исправления пропущенных значений воспользуемся медианными значениями по возрасту. Для этого нам потребуется выделить возрастные категории клиентов: в качестве границ категорий выбраны квантили (25%, 50% и 75%).

In [ ]:
age_statistics = data['dob_years'].describe()
age_statistics[4:7]

In [ ]:
# напишем функцию, которая принимает на вход возраст клиента и возвращает возрастную категорию
def age_group(age):
    if age <= age_statistics[4]: return 'до 34'
    elif age_statistics[4] < age <= age_statistics[5]: return '34-43'
    elif age_statistics[5] < age <= age_statistics[6]: return '43-53'
    else: return '53+'
    
data['age_group'] = data['dob_years'].apply(age_group)
data.head(2)

In [ ]:
# Найдем медианное значение трудового стажа для каждой возрастной группы.
medians_of_days_employed = data.groupby('age_group')['days_employed'].median()
round(medians_of_days_employed, 0)

In [ ]:
# заменим пропущенные значения трудового стажа на медиану из возрастных категорий
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '34-43'), 'days_employed'] = medians_of_days_employed[0]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '43-53'), 'days_employed'] = medians_of_days_employed[1]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == '53+'), 'days_employed'] = medians_of_days_employed[2]
data.loc[(data['days_employed'].isnull()) & (data['age_group'] == 'до 34'), 'days_employed'] = medians_of_days_employed[3]

### Вывод

In [ ]:
data.info()

Как мы видим, все строки теперь заполнены:
- обработаны пропущенные значения по ежемесячному доходу на основе медиан для каждого типа занятости;
- исправлены значения нулевого возраста на медианы соответствующего типа занятости;
- обработаны пропущенные значения в данных по трудовому стажу на основе медиан для каждой возрастной категории.

### Замена типа данных

Приведем еще раз для наглядности общую информацию по массиву данных.

In [ ]:
data.dtypes

На данный момент возраст клиентов записан вещественным числом – изменим тип значений этого признака с вещественного на целый.

In [ ]:
data['dob_years'] = data['dob_years'].astype('int')

In [ ]:
data.dtypes

### Вывод

Для замены типа данных был использован метод *'astype'*, так как мы уже обработали пропущенные значения (иначе можно было бы использовать *'to_numeric'*). Тип данных был изменен только для возраста (а не для ежемесячного дохода), так как ежемесячный доход все равно мы будем категоризировать и разбивать по группам.

### Обработка дубликатов

Перед поиском дубликатов приведем все данные по образованию клиентов к нижнему регистру. В противном случае, при поиске дубликатов значения "Среднее" и "среднее" будут определены, как 2 разных значения, что неверно.

In [ ]:
# сосчитаем всех клиентов по уровням образования
data['education'].value_counts()

In [ ]:
print('Количество неправильно заполненных ячеек со средним уровнем образования:', 772 + 711)
print('Доля ошибок от общего числа клиентов со средним образованием: {:.2%}'. format((772 + 711) / (13750 + 772 + 711)))

In [ ]:
# сделаем все буквы в значениях образования строчными
data['education'] = data['education'].str.lower()

# посмотрим, что получилось, одновременно посчитая количество клиентов по каждому уровню образования
data['education'].value_counts()

In [ ]:
# Посчитаем количество дубликатов в массиве.
print('Число дубликатов в массиве данных:', data.duplicated().sum())
print('Число полностью идентичных строк:', data.duplicated(keep = False).sum())
print('Доля дубликатов из общей длины массива: {:.2%}'.format(data.duplicated().sum() / len(data)))

In [ ]:
# Для наглядности выведем первые 5 строк полностью "идентичных" клиентов.
data[data.duplicated(keep = False)].sort_values('dob_years', ascending = False).head()

In [ ]:
# Удалим одинаковые строки.
data = data.drop_duplicates().reset_index(drop = True)

# Проверим число дубликатов.
print('Число дубликатов в массиве данных:', data.duplicated().sum())

### Вывод

1. Необходимо обратить внимание на метод заполнения ячеек с уровнем образования клиентов. На данный момент ячейки заполняются вручную, что приводит к значениям вроде "среднее", "Среднее" или "СРЕДНЕЕ". *На примере среднего образования*: всего были неправильно заполнены данные по **1483 клиентам (9,74% от общего числа клиентов со средним образованием)**. Необходимо ввести условие на нижний регистр и выпадающий список всех возможных вариантов уровня образования.
2. При обработке данных были найдены дубликаты в количестве **71 шт.**, что составляет **0,33% от всего числа клиентов в массиве данных**. Необходимо выяснить, связано ли появление дубликатов с технической ошибкой, или же ошибка носит человеческий фактор. При удалении дубликатов использовался метод *drop_duplicates()*, так как его целью является поиск идентичных строк, что нам и требовалось.

### Лемматизация

In [ ]:
# Импортируем библиотеку *pymystem3* и *collections*
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# Посчитаем все варианты целей кредита.
data['purpose'].value_counts()

Среди всех возможных целей получения кредита можно выделить всего 5 ключевых слов, которые отражают главную цель.

In [ ]:
# основные ключевые слова можно выделить вручную
categories = ["сдача", "коммерческий", "жилье", "образование", "свадьба", "недвижимость", "автомобиль"]

# проведем лемматизацию,
# одновременно заменив полученный список лемм в каждой строке на главное ключевое слово из списка категорий

def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

data['purpose_group'] = data['purpose'].apply(lemmatize)        
data.head(2)

In [ ]:
# Посчитаем все возможные категории целей кредита.
data['purpose_group'].value_counts()

Поскольку **жилье** тоже относится к **недвижимости**, то сделаем соответствующую замену. Кроме того, заменим категории по *покупке коммерческой недвижимости* или *покупке жилой недвижимости для сдачи в наем* на **инвестиционную цель** и снова посчитаем число категорий.

In [ ]:
data.loc[data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
data.loc[(data['purpose_group'] == 'коммерческий') | (data['purpose_group'] == 'сдача'), 'purpose_group'] = 'инвестиционная цель'
data['purpose_group'].value_counts()

### Вывод

Из цели получения кредита каждого клиента было выделено по одному ключевому слову:
- цель **на покупку своего автомобиля** заменяется на **автомобиль**
- цель **покупка жилой недвижимости** заменяется на **недвижимость**
- и т.д.

Подобное выделение ключевых слов из целей клиентов упрощает дальнейший анализ и сохраняет основной смысл цели получения кредита.

Все цели, содержащие производные слова от слова **жилье** (3809 чел.), были заменены на категорию **недвижимость** (цели с исходным ключевым словом насчитывают 5040 чел.), кроме целей по покупке коммерческой недвижимости (1311 чел.) или покупке жилой недвижимости для сдачи в наем (651 чел.).

Если обычная покупка жилой недвижимости относится к конечному потреблению, то приобретение коммерческой недвижимости или недвижимости для сдачи в наем относится к инвестиционным целям, т.е. этот кредит клиенты планируют не только вернуть, но вероятно еще и окупить. То есть клиенты планируют *в ближайшем будущем получать положительный денежный поток*. **Доля таких клиентов составляет 9,1%** от общего числа клиентов, поэтому ими не стоит пренебрегать, а стоит выделить отдельно.

### Категоризация данных

Ранее уже были созданы два вида категорий: один — по возрастам (*'age_group'*), другой — по целям (*'purpose_category'*). Но для ответа на поставленные вопросы потребуется провести еще одну категоризацию — по доходам. Для этого воспользуемся квантилями 25%, 50% и 75%.

In [ ]:
statistics = data['total_income'].describe()
statistics[4:7]

In [ ]:
# функция для определения категории доходов
def determine_income_group(income):
    if income <= statistics[4]: return 1
    elif statistics[4] < income <= statistics[5]: return 2
    elif statistics[5] < income <= statistics[6]: return 3
    else: return 4

# применим функцию к столбцу доходов
data['income_group'] = data['total_income'].apply(determine_income_group)
data.head()

### Вывод

#### Категоризация по возрастам

Категоризация по возрастам (*'age_group'*) была проведена при устранении пропущенных значений: в ней были использованы квантили (25%, 50% и 75%):

In [ ]:
data['age_group'].value_counts()

Больше всего клиентов в возрасте до 34 лет, их доля составляет 27,8%.

#### Категоризация по целям

Категоризация по целям (*'purpose_category'*) была проведена ранее при лемматизации.

In [ ]:
data['purpose_group'].value_counts()

Чаще всего клиенты берут кредит для приобретения недвижимости: доля таких клиентов из общего числа составляет 41,2%.

#### Категоризация по доходам

В качестве границ для категоризации доходов были использованы квантили (25%, 50% и 75%).

In [ ]:
data['income_group'].value_counts()

Другими словами:
- доход 5364 человек меньше или равен 107,6 тыс. рублей
- доход 5479 человек находится в диапазоне от 107,6 до 142,6 тыс. рублей
- доход 5247 человек находится в диапазоне от 142,6 до 195,8 тыс. рублей
- наконец, 5364 человек зарабатывают больше 195,8 тыс. рублей

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# Заменим ошибочные значения в количестве детей из следующего предположения:
# вместо 20 детей укажем 2-х, вместо -1 укажем 1.
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

# напишем функцию для категоризации клиентов: 1 — есть дети, 0 — нет детей.
def determine_children(children):
    if children > 0: return 1
    else: return 0

# добавим новый столбец с бинарным признаком в наш исходный массив
data['child_exist'] = data['children'].apply(determine_children)

Для ответа на поставленный вопрос даже не требуется формула Байеса, воспользуемся сводными таблицами.

Пусть:
- событие $А$ — существует задолженность по кредиту,
- событие $B$ — у клиента есть дети.

In [ ]:
# построим сводную таблицу
data_pivot = data.pivot_table(index = ['child_exist'], values = 'debt').round(3)
#data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot.head()

Тогда **вероятность задолженности при условии, что у клиента есть дети составляет:**

$$P(A|B) = 0.092$$

А **вероятность задолженности при условии, что у клиента нет детей составляет:**

$$P(A|\bar{B}) = 0.075$$

### Вывод

Вывод контринтуитивный: судя по имеющимся данным, наличие детей увеличивает вероятность задолженности по сравнению с отсутствием детей у клиента **на 1.7 пп.**: **9,2%** против **7,5%** соответственно.

Объяснить это можно следующим образом: при сложной материальной ситуации клиент предпочтет задолжать банку, чем не обеспечить своего ребенка всем необходимым.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждого вида семейного положения
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

### Вывод

Если клиент хотя бы раз в жизни был в браке (либо сейчас состоит в браке, либо супруг / супруга скончалась, либо развелся с супругом / супругой), то вероятность задолженности будет меньше **на 1.8-3.2 пп.**, чем у клиента, который не состоит в официальном браке. Видимо, жизнь в браке учит отдавать долги больше, чем жизнь вне брака.

Получившиеся вероятности, отсортированные по убыванию:
1. не женат / не замужем = **9,8%**
2. гражданский брак = **9,3%**
3. женат / замужем = **7,5%**
4. в разводе = **7,1%**
5. вдовец / вдова = **6,6%**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

### Вывод

Действительно, клиент зарабатывает больше, чем 75% других клиентов из выборки, он с наименьшей вероятностью окажется в должниках (вероятность всего **7,1%**). Однако удивительно, что на втором месте клиенты не из 50-75% группы, а клиенты из группы с наименьшим уровнем доходов: для них вероятность задолженноси составила всего **8,0%**. Объяснить это можно тем, что люди с наименьшими доходами лучше распоряжаются своими средствами и умеют контролировать свой бюджет, чем те, кто зарабатывает немногим больше.

Получившиеся вероятности задолженности, отсортированные по убыванию:
1. доход в диапазоне от 107,6 до 142,6 тыс. рублей в месяц = **8,8%**
2. доход в диапазоне от 142,6 до 195,8 тыс. рублей = **8,5%**
3. доход меньше или равен 107,6 тыс. руб. = **8,0%**
4. больше 195,8 тыс. рублей = **7,1%**

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

### Вывод

Получившиеся вероятности задолженности для каждой цели получения кредита, отсортированные по убыванию:
1. автомобиль - **9,4%**
2. образование - **9,2%**
3. свадьба - **8,0%**
4. инвестиционная цель - **7,7%**
5. недвижимость - **7,1%**

Самая низкая вероятность задолженности у клиентов, которые берут кредит для собственной недвижимости (покупка, строительство и т.д.). Люди настолько заинтересованы в собственном жилье, что платят даже ответственнее по сравнению с теми, кто берет кредит на инвестиционные цели (7,1% против 7,7%). Интересный факт: клиенты, берущие кредит на свадьбу, не самые безответственные плательщики из выборки – для них вероятность всего 8,0%. Хуже них платят только по образовательным и автокредитам.

### Шаг 4. Общий вывод

Портрет идеального клиента можно было бы описать следующим образом (описание будет для мужского пола, то же верно и для женского (зависимость задолженности по кредиту от пола клиента стоит рассмотреть в будущем)):
- *Женатый мужчина (либо ранее был женат), без детей, зарабатывает больше 196 тыс. рублей в месяц, цель получения кредита – приобретение собственного жилья.*

Такое описание говорит с одной стороны об умении нести ответственность в некоторой степени (в виде официального брака в настоящем или в прошлом), о высоком уровне достатка и некоторой доле эгоизма, поскольку человек не имеет детей. Таким образом, у него есть средства на выплату кредита, есть собственная материальная цель, но нет сдерживающих факторов в виде иждивенцев.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.